In [1]:
import os
os.chdir("D:\PulpitE\FPL_ML")

In [2]:
from knapsack_problem.knapsack import best_full_teams
from knapsack_problem.player import Player
import pandas as pd
import math

In [3]:
DATA_PATH = "predictions/preds_next_season.csv"

In [4]:
df = pd.read_csv(DATA_PATH).rename(columns={"Pred": "Prediction"})

In [5]:
df.columns

Index(['Name_original', 'GW', 'Season', 'Team', 'Opponent', 'Was_home',
       'Team_rating', 'Opp_rating', 'FPL_pos', 'Price', 'Prediction'],
      dtype='object')

In [6]:
df = df[df["GW"] == 2]

In [7]:
df = df[df["Prediction"] > 5.6]

In [8]:
df.shape, df[df["FPL_pos"] == "GK"].shape, df[df["FPL_pos"] == "DEF"].shape, df[df["FPL_pos"] == "MID"].shape, df[df["FPL_pos"] == "FWD"].shape

((39, 11), (0, 11), (9, 11), (18, 11), (12, 11))

In [9]:
df.head()

,Name_original,GW,Season,Team,Opponent,Was_home,Team_rating,Opp_rating,FPL_pos,Price,Prediction
8,Erling Haaland,2.0,2023-24,Manchester City,Newcastle Utd,1.0,2077.251953,1875.999878,FWD,140.0,10.486730
242,Marcus Rashford,2.0,2023-24,Manchester Utd,Tottenham,0.0,1873.631104,1815.873291,MID,90.0,7.867932
309,Julián Álvarez,2.0,2023-24,Manchester City,Newcastle Utd,1.0,2077.251953,1875.999878,FWD,65.0,7.629467
329,Gabriel Martinelli Silva,2.0,2023-24,Arsenal,Crystal Palace,0.0,1919.368774,1757.395630,MID,80.0,7.550077
372,John McGinn,2.0,2023-24,Aston Villa,Everton,1.0,1827.994629,1708.234497,MID,55.0,7.420167


In [10]:
playersDB = []
for i, row in df.iterrows():
    playersDB.append(Player(row.Name_original, row.FPL_pos, row.Price, round(row.Prediction, 2), row.Team))

In [11]:
# 2 example Goalkeepers
playersDB.append(Player("GK1", "GK", 5, 4, "Manchester Utd"))
playersDB.append(Player("GK2", "GK", 4, 3, "Nott'ham Forest"))

In [12]:
possible_formations = [
    [3, 4, 3],
    [3, 5, 2],
    [4, 3, 3],
    [4, 4, 2],
    [4, 5, 1],
    [5, 3, 2],
    [5, 4, 1],
]

In [13]:
budget = 1000

In [14]:
%%time
best_team = best_full_teams(playersDB, possible_formations, budget)

With formation [3, 5, 2]: 78.8  | (price = 800.0)
(GK1, GK, 5, 4, Manchester Utd) - (fixtures: 1.0)
(Pervis Estupiñán, DEF, 50.0, 7.05, Brighton) - (fixtures: 1.0)
(Ibrahima Konaté, DEF, 50.0, 6.29, Liverpool) - (fixtures: 1.0)
(Trent Alexander-Arnold, DEF, 80.0, 6.27, Liverpool) - (fixtures: 1.0)
(Marcus Rashford, MID, 90.0, 7.87, Manchester Utd) - (fixtures: 1.0)
(Gabriel Martinelli Silva, MID, 80.0, 7.55, Arsenal) - (fixtures: 1.0)
(John McGinn, MID, 55.0, 7.42, Aston Villa) - (fixtures: 1.0)
(Mohamed Salah, MID, 125.0, 7.25, Liverpool) - (fixtures: 1.0)
(Alexis Mac Allister, MID, 60.0, 6.98, Liverpool) - (fixtures: 1.0)
(Erling Haaland, FWD, 140.0, 10.49, Manchester City) - (fixtures: 1.0)
(Julián Álvarez, FWD, 65.0, 7.63, Manchester City) - (fixtures: 1.0)


With formation [3, 4, 3]: 78.59  | (price = 800.0)
(GK1, GK, 5, 4, Manchester Utd) - (fixtures: 1.0)
(Pervis Estupiñán, DEF, 50.0, 7.05, Brighton) - (fixtures: 1.0)
(Ibrahima Konaté, DEF, 50.0, 6.29, Liverpool) - (fixtures: 1.

In [15]:
best_team[0][1]

78.8

In [16]:
def best_team_to_df(best_team_squad, df, custom_fpl_order):
    names = [p.name for p in best_team_squad]
    
    # Convert "FPL_pos" column to categorical with custom order
    df["FPL_pos"] = pd.Categorical(df["FPL_pos"], categories=custom_fpl_order, ordered=True)
    
    # Filter by names and sort by custom ordered "FPL_pos" and "Price"
    return df[df["Name_original"].isin(names)].sort_values(by=["FPL_pos", "Price"], ascending=False)

# Define your custom order for FPL_pos
custom_fpl_order = ["FWD", "MID", "DEF", "GK"]

In [17]:
best_squad = best_team_to_df(best_team[0][2], df, custom_fpl_order)
best_squad

,Name_original,GW,Season,Team,Opponent,Was_home,Team_rating,Opp_rating,FPL_pos,Price,Prediction
968,Trent Alexander-Arnold,2.0,2023-24,Liverpool,Bournemouth,1.0,1946.899780,1660.637939,DEF,80.0,6.267623
500,Pervis Estupiñán,2.0,2023-24,Brighton,Wolves,0.0,1828.169067,1719.037964,DEF,50.0,7.050610
937,Ibrahima Konaté,2.0,2023-24,Liverpool,Bournemouth,1.0,1946.899780,1660.637939,DEF,50.0,6.293412
443,Mohamed Salah,2.0,2023-24,Liverpool,Bournemouth,1.0,1946.899780,1660.637939,MID,125.0,7.250062
242,Marcus Rashford,2.0,2023-24,Manchester Utd,Tottenham,0.0,1873.631104,1815.873291,MID,90.0,7.867932
329,Gabriel Martinelli Silva,2.0,2023-24,Arsenal,Crystal Palace,0.0,1919.368774,1757.395630,MID,80.0,7.550077
515,Alexis Mac Allister,2.0,2023-24,Liverpool,Bournemouth,1.0,1946.899780,1660.637939,MID,60.0,6.981152
372,John McGinn,2.0,2023-24,Aston Villa,Everton,1.0,1827.994629,1708.234497,MID,55.0,7.420167
8,Erling Haaland,2.0,2023-24,Manchester City,Newcastle Utd,1.0,2077.251953,1875.999878,FWD,140.0,10.486730
309,Julián Álvarez,2.0,2023-24,Manchester City,Newcastle Utd,1.0,2077.251953,1875.999878,FWD,65.0,7.629467


In [18]:
best_squad.to_csv("predictions/best_squad.csv")